In [1]:
import matplotlib.pyplot as plt
import re
import numpy as np # linear algebra
import random
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from pandas import datetime
import math, time
import itertools
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

%matplotlib notebook

In [2]:
SEQUENCE_LENGTH=3227
STOCKS = ['LLY', 'BMY', 'JNJ', 'ABBV', 'MRK', 'ZTS', 'PRGO', 'ELAN', 'TEVA', 'NVO']

In [3]:
def get_vector(x):
    x = re.sub(r'(\(|\))', '', x)
    x = x.split(',')
    return np.array([float(x1.strip()) for x1 in x])

### Obtain Tweet Vector in Numpy array format, Remove unnecessary attributes

In [4]:
df_tweets_full = pd.read_csv("/content/tweet_stock_data.csv", index_col=False, parse_dates=['Date'])
df_tweets_full['Tweet_Vector'] = df_tweets_full['Tweet Vector'].apply(lambda x: get_vector(x))
df_tweets_full.drop(columns=['Unnamed: 0', 'Tweet Vector'], inplace=True)

### Group by Stock and Sort by Date

In [5]:
df_tweets_full = df_tweets_full.groupby('Stock').apply(lambda x: x.sort_values('Date'))

### Find average of the tweet vector based on Date

In [6]:
df = pd.DataFrame()
for stock in STOCKS:
  df_tweets = df_tweets_full.loc[df_tweets_full['Stock']==stock, ['Date', 'Tweet_Vector']]
  df_tweets = df_tweets.groupby('Date')['Tweet_Vector'].apply(np.mean).reset_index()
  df_tweets['Stock'] = stock
  df = df.append(df_tweets, ignore_index=True)

### Prepare dataset using Stock Price 

##### 1. Load the dataset
##### 2. The output format must be tweet vector and stock prices
##### 3. Populate the stock prices based on the date and stock name
##### 4. Fill the missing values with zero vector

In [7]:
df_stocks = pd.read_csv("/content/stock_data.csv", index_col=False)

df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], infer_datetime_format=True)

df_stocks.drop(columns=['Unnamed: 0'], inplace=True)

df_stocks.head(5)

,Stock,Date,Open,Close,High,Low,Volume
0,LLY,2018-01-02,84.459999,84.680000,85.000000,84.059998,3456700
1,LLY,2018-01-03,84.739998,85.139999,85.419998,84.269997,3078400
2,LLY,2018-01-04,85.279999,85.519997,85.800003,84.739998,2462500
3,LLY,2018-01-05,86.089996,86.570000,86.750000,85.930000,2946300
4,LLY,2018-01-08,86.690002,86.129997,86.709999,85.400002,3251400


In [8]:
df_stock = df_stocks.drop(['Open', 'High', 'Low', 'Volume'], axis=1)
scaler = MinMaxScaler(feature_range=(-1, 1))
df_stock['Close'] = scaler.fit_transform(df_stock['Close'].values.reshape(-1,1))
print(df_stock.head(5))

  Stock       Date     Close
0   LLY 2018-01-02  0.062301
1   LLY 2018-01-03  0.068525
2   LLY 2018-01-04  0.073666
3   LLY 2018-01-05  0.087871
4   LLY 2018-01-08  0.081918


In [9]:
print(df.tail(5))

           Date                                       Tweet_Vector Stock
1474 2019-09-23  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   NVO
1475 2019-11-18  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   NVO
1476 2020-01-17  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   NVO
1477 2020-02-05  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   NVO
1478 2020-10-05  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   NVO


In [10]:
zero_vector = np.zeros(3228, dtype = float)
dfinal = df_stock.merge(df, on=['Date', 'Stock'], how = 'left')
dfinal.tail(10)

,Stock,Date,Close,Tweet_Vector
5229,NVO,2020-02-13,-0.234526,NaN
5230,NVO,2020-02-14,-0.224921,NaN
5231,NVO,2020-02-18,-0.216262,NaN
5232,NVO,2020-02-19,-0.215586,NaN
5233,NVO,2020-02-20,-0.224109,NaN
5234,NVO,2020-02-21,-0.219509,NaN
5235,NVO,2020-02-24,-0.254279,NaN
5236,NVO,2020-02-25,-0.263208,NaN
5237,NVO,2020-02-26,-0.256714,NaN
5238,NVO,2020-02-27,-0.278496,NaN


In [11]:
for row in dfinal.loc[dfinal.Tweet_Vector.isnull(), 'Tweet_Vector'].index:
    dfinal.at[row, 'Tweet_Vector'] = zero_vector
dfinal.tail(5)

,Stock,Date,Close,Tweet_Vector
5234,NVO,2020-02-21,-0.219509,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5235,NVO,2020-02-24,-0.254279,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5236,NVO,2020-02-25,-0.263208,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5237,NVO,2020-02-26,-0.256714,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5238,NVO,2020-02-27,-0.278496,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [12]:
df = dfinal.loc[dfinal['Stock']=='LLY', ['Date', 'Close', 'Tweet_Vector']]
scaler = MinMaxScaler(feature_range=(-1, 1))
df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))
df = df.set_index('Date')
df.head()

,Close,Tweet_Vector
Date,,
2018-01-02,-0.713700,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2018-01-03,-0.701121,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2018-01-04,-0.690730,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2018-01-05,-0.662018,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2018-01-08,-0.674050,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [15]:
def load_data(dfinal, stock):
    df = dfinal.loc[dfinal['Stock']==stock, ['Date', 'Close', 'Tweet_Vector']]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))
    df = df.set_index('Date')
    
    df_close = df.drop(['Tweet_Vector'], axis=1)
    label = df_close.to_numpy()

    df_vector = df.drop(['Close'], axis=1)
    df_vector['Tweet_Vector'] = df_vector['Tweet_Vector'].to_list()
    vector = []
    for tweet in df_vector['Tweet_Vector']:
      row = []
      for word in tweet:
        row.append([word])
      vector.append(row)
    data = np.array(vector)

    test_set_size = int(np.round(0.2*data.shape[0]))
    train_set_size = data.shape[0] - (test_set_size)
    
    x_train = data[:train_set_size,:-1,:]
    y_train = label[:train_set_size,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = label[train_set_size:,:]
    
    return x_train, y_train, x_test, y_test


In [16]:
x_data = False

for stock in STOCKS:
    x_train_, y_train_, x_test_, y_test_ = load_data(dfinal, stock)
    if not x_data:
        x_data = True
        x_train = x_train_
        y_train = y_train_
        x_test = x_test_
        y_test = y_test_
    else:
        x_train = np.concatenate((x_train, x_train_))
        y_train = np.concatenate((y_train, y_train_))
        x_test = np.concatenate((x_test, x_test_))
        y_test = np.concatenate((y_test, y_test_))
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (4195, 3227, 1)
y_train.shape =  (4195, 1)
x_test.shape =  (1044, 3227, 1)
y_test.shape =  (1044, 1)


In [17]:
# make training and test sets in torch
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [18]:
# Build model

input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
look_back=SEQUENCE_LENGTH

# Here we define our model as a class
class StockRebels(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(StockRebels, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # lstm layer1
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Initialize
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # One time step
        out, (hn, cn) = self.lstm1(x, (h0.detach(), c0.detach()))
        
        out = self.fc(out[:, -1, :]) 
        return out
    
model = StockRebels(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

loss_fn = torch.nn.MSELoss(size_average=True)

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

StockRebels(
  (lstm1): LSTM(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)
10
torch.Size([128, 1])
torch.Size([128, 32])
torch.Size([128])
torch.Size([128])
torch.Size([128, 32])
torch.Size([128, 32])
torch.Size([128])
torch.Size([128])
torch.Size([1, 32])
torch.Size([1])


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [19]:
batch_size = 128

train = torch.utils.data.TensorDataset(x_train,y_train)
test = torch.utils.data.TensorDataset(x_test,y_test)

train_loader = torch.utils.data.DataLoader(dataset=train, 
                                           batch_size=batch_size, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [ ]:
# Train model

num_epochs = 10
hist = np.zeros((num_epochs, 2))

for t in range(num_epochs):
    for x_train_, y_train_ in train_loader:
        optimiser.zero_grad()
        # Forward pass
        y_train_pred_ = model(x_train_)
        loss = loss_fn(y_train_pred_, y_train_)
        # Backward pass
        loss.backward()
        # Update parameters
        optimiser.step()    
    # Loss calculation
    y_train_pred = model(x_train)
    loss_train = loss_fn(y_train_pred, y_train)
    y_test_pred = model(x_test)
    loss_test = loss_fn(y_test_pred, y_test)
    print(f"Epoch {t}, Train MSE: {loss.item()}, Test MSE: {loss_test.item()}")
    hist[t][0] = loss.item()
    hist[t][1] = loss_test.item()

In [ ]:
# make predictions  
y_test_pred = model(x_test)
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred.detach().numpy()[:,0]))
print('Test Score: %.4f RMSE' % (testScore))